In [7]:
!pip install pretty_midi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 63.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 KB 6.8 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592305 sha256=2c0e1d7d26a90074ecfea89ee3e99b0ec7c87b33c2c0c888e2937112e1031fd4
  Stored in directory: /root/.cache/pip/wheels/96/4a/bd/ea2d2c62f684a0bcce6e6d277cb69e4463b5fd372c422c2481
Successfully built pretty_midi


In [2]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True) #mounts the google drive where the dataset is located

Mounted at /content/gdrive


In [40]:
import pretty_midi
import os
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [41]:
# Step 1: Collect MIDI files
midi_folder = "/content/gdrive/MyDrive/maestro3"
midi_files = [os.path.join(midi_folder, f) for f in os.listdir(midi_folder) if f.endswith('.mid') or f.endswith('.midi')]

batch_size = 32
num_batches = len(midi_files) // batch_size + 1


In [44]:
num_batches = int(np.ceil(len(midi_files) / batch_size))
max_length = 8000  # Set the maximum length for the piano roll arrays

for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(midi_files))
    batch_files = midi_files[start_idx:end_idx]
    midi_data = []
    for midi_file in batch_files:
        try:
            midi = pretty_midi.PrettyMIDI(midi_file)
            piano_roll = midi.get_piano_roll(fs=100)
            if piano_roll.shape[1] == 0:
                continue  # Skip over MIDI files with an empty piano roll array
            # Truncate the piano roll array to a maximum length of 8000
            if piano_roll.shape[1] > max_length:
                piano_roll = piano_roll[:, :max_length]
            midi_data.append(piano_roll)
        except Exception as e:
            print(e)
    # Pad the piano roll arrays to a fixed length
    padded_midi_data = np.zeros((len(midi_data), 128, max_length))
    for j, piano_roll in enumerate(midi_data):
        padded_midi_data[j, :, :piano_roll.shape[1]] = piano_roll
    midi_data = padded_midi_data


In [45]:
# Step 3: Preprocess MIDI data
midi_data = np.clip(midi_data, 0, 1)  # normalize data to [0, 1] range
print(midi_data.shape)

midi_data = np.round(midi_data)  # quantize data to 0 or 1 values
print(midi_data.shape)

midi_data = np.transpose(midi_data, (0, 2, 1))  # transpose data to (samples, time, pitch) format
print(midi_data.shape)



(10, 128, 8000)
(10, 128, 8000)
(10, 8000, 128)


In [46]:
# Step 4: Split data into training and validation sets
split_index = int(0.8 * len(midi_data))
train_data = midi_data[:split_index]
val_data = midi_data[split_index:]


In [10]:
# Step 5: Save preprocessed data
# np.savez_compressed("preprocessed_data.npz", train_data=train_data, val_data=val_data)

NameError: ignored

In [47]:
from keras.models import Sequential
from keras.layers import LSTM, Dense



---



In [48]:
x_train = train_data  # input sequences

print(x_train.shape)

(8, 8000, 128)


In [55]:
model = Sequential()
model.add(LSTM(128, input_shape=(None, 128), return_sequences=True))  # Modified LSTM layer
model.add(Dense(128, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])


In [57]:
num_epochs = 100
batch_size = 32
model.fit(train_data, train_data, batch_size=batch_size, epochs=num_epochs)


Epoch 1/100
1/1 [==============================] - 0s 484ms/step - loss: 23.8786 - accuracy: 0.0276
Epoch 2/100
1/1 [==============================] - 0s 316ms/step - loss: 23.8731 - accuracy: 0.0275
Epoch 3/100
1/1 [==============================] - 0s 338ms/step - loss: 23.8698 - accuracy: 0.0275
Epoch 4/100
1/1 [==============================] - 0s 300ms/step - loss: 23.8660 - accuracy: 0.0275
Epoch 5/100
1/1 [==============================] - 0s 313ms/step - loss: 23.8607 - accuracy: 0.0274
Epoch 6/100
1/1 [==============================] - 0s 319ms/step - loss: 23.8571 - accuracy: 0.0274
Epoch 7/100
1/1 [==============================] - 0s 272ms/step - loss: 23.8570 - accuracy: 0.0274
Epoch 8/100
1/1 [==============================] - 0s 247ms/step - loss: 23.8577 - accuracy: 0.0273
Epoch 9/100
1/1 [==============================] - 0s 239ms/step - loss: 23.8568 - accuracy: 0.0271
Epoch 10/100
1/1 [==============================] - 0s 249ms/step - loss: 23.8549 - accuracy: 0.0271